### **RunnablePassthrough**

- 입력을 그대로 반환하는 Runnable

- 체인 내에서 원본 데이터를 유지할 때 유용

- 기본적으로 Dict 형태로 전달하지만 RunnablePassthrough를 사용하면 Dict 형태로 전달하지 않아도 됨

- 데이터를 받아서 전달 해준다 

`RunnablePassthrough` 는 `runnable` 객체이며, `runnable` 객체는 `invoke()` 메소드를 사용하여 별도 실행이 가능

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from IPython.display import display, Markdown

### **기본 사용 방법**

In [52]:
llm = ChatOpenAI(model = 'gpt-4o')

prompt = PromptTemplate.from_template(
    "You are a helpful assistant. Answer the question: {question}"
)

chain = ({'question' : RunnablePassthrough()} 
        | prompt
        | llm 
        | StrOutputParser()
)

chain.invoke('안녕 너는 어떤것들을 도와 줄 수 있어?')

'안녕하세요! 저는 다양한 분야에서 도움을 드릴 수 있습니다. 예를 들어 정보 검색, 질문에 대한 답변, 언어 번역, 일정 관리, 아이디어 제안, 기술적인 문제 해결 등을 도와드릴 수 있습니다. 필요하신 것이 있다면 언제든지 말씀해 주세요!'

### **RAG에 활용**

In [40]:
embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')

vectordb = FAISS.from_texts(
    ["에스티이노베이션은 리서치, 인공지능, 데이터 분석을 통한 여러 인사이트를 발굴하는 기업입니다", 
     "에스티이노베이션은 2023년 매출 18억, 2024년에는 매출 20억을 달성하였습니다", 
     "에스티이노베이션은 20명의 직원이 있습니다.",
     '현재는 2025년 3월입니다.'],
    embeddings
)

retriver = vectordb.as_retriever()

prompt = PromptTemplate.from_template(
    '''
    사용자의 질문에 대해 답변을 작성하세요. 
    검색된 context를 바탕으로 답변을 작성하세요.
    
    # Context:
    {context}

    # question: 
    {question}
    
  '''
)

llm = ChatOpenAI(model = 'gpt-4o')  

chain = (
   {'context' : retriver, 'question' : RunnablePassthrough()}
   | prompt
   | llm
    | StrOutputParser()
)

response = chain.invoke('에스티이노베이션이라는 회사가 2025년에는 어떤 방향으로 나아가야 더 좋은 기업이 될 수 있을지 컨설팅해줘')

display(Markdown(response))

에스티이노베이션은 리서치, 인공지능, 데이터 분석을 통해 여러 인사이트를 발굴하는 기업으로, 빠르게 변화하는 기술 환경 속에서 경쟁력을 유지하고 발전할 수 있는 여러 전략을 고려할 수 있습니다. 다음은 2025년에 에스티이노베이션이 나아가야 할 방향에 대한 제안입니다:

1. **지속적인 기술 혁신**: 인공지능과 데이터 분석 기술은 끊임없이 발전하고 있습니다. 최신 기술 트렌드를 지속적으로 모니터링하고, 필요한 경우 신기술을 빠르게 도입하여 서비스와 제품의 품질을 향상시키는 것이 중요합니다.

2. **고급 인력 확보 및 교육**: 2023년과 2024년의 안정적인 매출 성장과 20명의 직원 규모를 감안할 때, 우수한 인재를 유치하고 현재 인력을 지속적으로 교육하는 것이 필요합니다. 이를 통해 기업의 전문성을 강화할 수 있습니다.

3. **맞춤형 솔루션 제공**: 고객의 필요와 시장 트렌드에 맞춘 맞춤형 솔루션을 개발하여 차별화된 가치를 제공할 수 있습니다. 이를 위해서는 고객과의 긴밀한 커뮤니케이션이 필수적입니다.

4. **새로운 시장 개척**: 국내외의 새로운 시장을 탐색하고 진출함으로써 성장 기회를 모색할 수 있습니다. 특히 인공지능 및 데이터 분석이 필요한 다양한 산업에 진출할 수 있는 기회를 검토해 보세요.

5. **파트너십 및 협력 강화**: 다른 기업이나 연구 기관과의 협력 관계를 강화하여 시너지를 창출할 수 있습니다. 공동 프로젝트나 연구 개발을 통해 더 큰 혁신을 이룰 수 있습니다.

이러한 전략들을 종합적으로 고려하여 실행 계획을 수립하면 에스티이노베이션이 더 큰 성장을 이루고, 더욱 경쟁력 있는 기업이 될 수 있을 것입니다.